## Modeling

Multiple machine learning models were used in this project, each with their unique capabilities to handle classification tasks. The models include Gradient Boosting, AdaBoost, Support Vector Classifier (SVC), Logistic Regression, Random Forest, and a Deep Neural Network.  

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, RocCurveDisplay
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.metrics import Recall
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [17]:
credit = pd.read_csv('../data/train_cleaned.csv')
credit.head()

,id,is_delinquent,length_of_credit,number_of_delinquent_months,average_delinquency_rate,3mo_delinquency,6mo_delinquency,12mo_delinquency,flag_own_car,flag_own_realty,...,name_family_status,name_housing_type,flag_mobil,flag_work_phone,flag_phone,flag_email,occupation_type,cnt_fam_members,age,years_employed
0,5008804,0,16,2,0.125000,0,0,0,Y,Y,...,Civil marriage,Rented apartment,1,1,0,0,missing,2.0,32,12
1,5008805,0,15,2,0.133333,0,0,0,Y,Y,...,Civil marriage,Rented apartment,1,1,0,0,missing,2.0,32,12
2,5008806,0,30,7,0.233333,0,0,1,Y,Y,...,Married,House / apartment,1,0,0,0,Security staff,2.0,58,3
3,5008808,1,5,2,0.400000,1,1,1,N,Y,...,Single / not married,House / apartment,1,0,1,1,Sales staff,1.0,52,8
4,5008809,0,5,0,0.000000,0,0,0,N,Y,...,Single / not married,House / apartment,1,0,1,1,Sales staff,1.0,52,8


In [18]:
credit.columns

Index(['id', 'is_delinquent', 'length_of_credit',
       'number_of_delinquent_months', 'average_delinquency_rate',
       '3mo_delinquency', '6mo_delinquency', '12mo_delinquency',
       'flag_own_car', 'flag_own_realty', 'cnt_children', 'amt_income_total',
       'name_income_type', 'name_education_type', 'name_family_status',
       'name_housing_type', 'flag_mobil', 'flag_work_phone', 'flag_phone',
       'flag_email', 'occupation_type', 'cnt_fam_members', 'age',
       'years_employed'],
      dtype='object')

In [19]:
X = credit.drop(columns=[
    'id', 'is_delinquent', 'number_of_delinquent_months', 'average_delinquency_rate', 
    '3mo_delinquency', '6mo_delinquency', '12mo_delinquency'])

y = credit['is_delinquent']

In [20]:
cat_cols = X_train.select_dtypes(include='object').columns
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

## Train-test(Validation) split

In [21]:
X = credit.drop(columns=[
    'id', 'is_delinquent', 'number_of_delinquent_months', 'average_delinquency_rate', 
    '3mo_delinquency', '6mo_delinquency', '12mo_delinquency'])

y_1 = credit['is_delinquent']
y_2 = credit['3mo_delinquency']
y_3 = credit['6mo_delinquency']
y_4 = credit['12mo_delinquency']


X_train, X_val, y_train_1, y_val_1 = train_test_split(X, y_1, test_size=0.2, random_state=42)

X_train, X_val, y_train_2, y_val_2 = train_test_split(X, y_2, test_size=0.2, random_state=42)
X_train, X_val, y_train_3, y_val_3 = train_test_split(X, y_3, test_size=0.2, random_state=42)

X_train, X_val, y_train_4, y_val_4 = train_test_split(X, y_4, test_size=0.2, random_state=42)

## One-Hot-Encoding and Standard Scaling variables with Column Transform

In [22]:
cat_cols = X_train.select_dtypes(include='object').columns
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

sc = StandardScaler()

oh = OneHotEncoder(
    categories='auto', 
    drop='first', 
    sparse_output=False, 
    dtype='int',
    handle_unknown = 'ignore'
)

ct = ColumnTransformer(
    transformers=[
        ('oh', oh, cat_cols),
        ('sc', sc, num_cols)
    ]
)

X_train_ct = ct.fit_transform(X_train)
X_val_ct = ct.transform(X_val)

## Fit and scoring function

In [40]:
data_sets = [("is_delinquent", y_train_1, y_val_1), 
             ("3mo_delinquency", y_train_2, y_val_2), 
             ("6mo_delinquency", y_train_3, y_val_3)]

def fit_and_score(gs, X_train, y_train, X_val, y_val, name):
    print(f"\nPerforming grid search and scoring on dataset: {name}\n{'-'*50}")
    gs.fit(X_train, y_train)
    print(f'Training score on {name} : {gs.score(X_train, y_train)}')
    print(f'Testing score on {name} : {gs.score(X_val, y_val)}')
    best_params = gs.best_params_
    print(f"Best parameters for {name}:", best_params)
    return best_params


def fit_and_score_no_param(gs, X_train, y_train, X_val, y_val, name):
    print(f"\nPerforming grid search and scoring on dataset: {name}\n{'-'*50}")
    gs.fit(X_train, y_train)
    print(f'Training score on {name} : {gs.score(X_train, y_train)}')
    print(f'Testing score on {name} : {gs.score(X_val, y_val)}')
    return 



## Target Column Distributions

Predicting the classes with 3 different targets

In [41]:
print(credit['is_delinquent'].value_counts(normalize=True))
print(credit['3mo_delinquency'].value_counts(normalize=True))
print(credit['6mo_delinquency'].value_counts(normalize=True))

0    0.757308
1    0.242692
Name: is_delinquent, dtype: float64
0    0.589633
1    0.410367
Name: 3mo_delinquency, dtype: float64
0    0.526401
1    0.473599
Name: 6mo_delinquency, dtype: float64


## Gradient Boosting with default parameters

In [48]:
gb = GradientBoostingClassifier()

for name, y_train, y_val in data_sets:
    fit_and_score_no_param(gb, X_train_ct, y_train, X_val_ct, y_val, name)



Performing grid search and scoring on dataset: is_delinquent
--------------------------------------------------
Training score on is_delinquent : 0.778815196394076
Testing score on is_delinquent : 0.7633612363168062

Performing grid search and scoring on dataset: 3mo_delinquency
--------------------------------------------------
Training score on 3mo_delinquency : 0.758813586606568
Testing score on 3mo_delinquency : 0.7546683837733419

Performing grid search and scoring on dataset: 6mo_delinquency
--------------------------------------------------
Training score on 6mo_delinquency : 0.7706455247907277
Testing score on 6mo_delinquency : 0.766580811332904


## Gradient Boosting with GridSearchCV

In [52]:
pgrids = {
    'learning_rate': [.15, .175, 0.2],
    'n_estimators': [285, 290, 295],
    'max_depth': [8, 9, 10]
}
    
gb = GradientBoostingClassifier()
gs = GridSearchCV(gb, param_grid=pgrids)

for name, y_train, y_val in data_sets:
    fit_and_score_no_param(gs, X_train_ct, y_train, X_val_ct, y_val, name)


Performing grid search and scoring on dataset: is_delinquent
--------------------------------------------------


KeyboardInterrupt: 

## Ada Boost with GridSearchCV

In [ ]:
%%time
tree = DecisionTreeClassifier(random_state=123)#, max_depth=1
ada = AdaBoostClassifier(estimator=tree, random_state=42)

pgrids_ada = {
    'learning_rate': [2.25, 2.5, 2.7],
    'n_estimators': [250, 300, 350],
    'estimator__max_depth': [71, 81, 91, 101, None],
    # 'estimator__min_samples_leaf': [7, 8, 9],
    'estimator__max_features': ['auto', 'sqrt', 'log2']
}


gs_ada_2 = GridSearchCV(ada, param_grid=pgrids_ada, cv=5)
gs_ada_2.fit(X_train_ct, y_train_3)

print(gs_ada.score(X_train_ct, y_train_3))
print(gs_ada.score(X_val_ct, y_val_3))  


## Support Vector Classifier

In [42]:
svc = SVC(C=1, kernel='rbf', degree = 1)

for name, y_train, y_val in data_sets:
    fit_and_score_no_param(svc, X_train_ct, y_train, X_val_ct, y_val, name)


Performing grid search and scoring on dataset: is_delinquent
--------------------------------------------------
Training score on is_delinquent : 0.7627978106889891
Testing score on is_delinquent : 0.7548293625241468

Performing grid search and scoring on dataset: 3mo_delinquency
--------------------------------------------------
Training score on 3mo_delinquency : 0.7787749517063748
Testing score on 3mo_delinquency : 0.7598197037990985

Performing grid search and scoring on dataset: 6mo_delinquency
--------------------------------------------------
Training score on 6mo_delinquency : 0.788554410817772
Testing score on 6mo_delinquency : 0.7670637475853187


## Logistic Regression

In [45]:
param_grid = {
    'C':  np.linspace(1, 10,5),
    'penalty': ['l2'],
    'solver': ['lbfgs'],
    'max_iter': [500,1000]
    
}

gs = GridSearchCV(
estimator= LogisticRegression(),
param_grid=param_grid)


for name, y_train, y_val in data_sets:
    fit_and_score(gs, X_train_ct, y_train, X_val_ct, y_val, name)


Performing grid search and scoring on dataset: is_delinquent
--------------------------------------------------
Training score on is_delinquent : 0.7627575660012879
Testing score on is_delinquent : 0.754507405022537
Best parameters for is_delinquent: {'C': 10.0, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}

Performing grid search and scoring on dataset: 3mo_delinquency
--------------------------------------------------
Training score on 3mo_delinquency : 0.749637797810689
Testing score on 3mo_delinquency : 0.7435608499678042
Best parameters for 3mo_delinquency: {'C': 10.0, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}

Performing grid search and scoring on dataset: 6mo_delinquency
--------------------------------------------------
Training score on 6mo_delinquency : 0.7563184159690921
Testing score on 6mo_delinquency : 0.7549903412749517
Best parameters for 6mo_delinquency: {'C': 1.0, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}


## Random Forest

In [50]:
import pickle

param_grid = {
    'n_estimators': [300,400],
    'max_depth': [None],
    'min_samples_split': [5,8],
    'min_samples_leaf': [3, 4],
    'max_features': ['sqrt']
}

rf = RandomForestClassifier()

gs = GridSearchCV(estimator=rf, param_grid=param_grid)

for name, y_train, y_val in data_sets:
    fit_and_score(gs, X_train_ct, y_train, X_val_ct, y_val, name)

    
with open("random_forest_model.pkl", "wb") as file:
    pickle.dump(gs, file)



Performing grid search and scoring on dataset: is_delinquent
--------------------------------------------------
Training score on is_delinquent : 0.8697279459111398
Testing score on is_delinquent : 0.78783000643915
Best parameters for is_delinquent: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 400}

Performing grid search and scoring on dataset: 3mo_delinquency
--------------------------------------------------
Training score on 3mo_delinquency : 0.8578155183515775
Testing score on 3mo_delinquency : 0.7933032839665164
Best parameters for 3mo_delinquency: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 300}

Performing grid search and scoring on dataset: 6mo_delinquency
--------------------------------------------------
Training score on 6mo_delinquency : 0.8566081777205409
Testing score on 6mo_delinquency : 0.7905666452028333
Best parameters for 6mo_delinquency: {'m

## Dense Neural Network

In [4]:
def dnn(X_train, y_train, X_val, y_val):

    model = Sequential()
    model.add(Dense(256, input_dim=X_train_ct.shape[1], activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy', 'Recall'])

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(
    X_train_ct, 
    y_train_1, 
    epochs=50, 
    batch_size=32, 
    validation_data=(X_val, y_val),
    callbacks = es
    )

    return model

In [5]:
# Model did worse with dropouts, l2 regularizer
# Performed similar with extra hidden layers

In [18]:
is_delinq = dnn(X_train_ct, y_train_1, X_val_ct, y_val_1)

Epoch 1/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4989 - accuracy: 0.7588 - recall: 0.0910 - val_loss: 0.4975 - val_accuracy: 0.7527 - val_recall: 0.0045
Epoch 2/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4848 - accuracy: 0.7669 - recall: 0.2183 - val_loss: 0.4965 - val_accuracy: 0.7534 - val_recall: 0.0883
Epoch 3/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4806 - accuracy: 0.7691 - recall: 0.2596 - val_loss: 0.4944 - val_accuracy: 0.7576 - val_recall: 0.3245
Epoch 4/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4754 - accuracy: 0.7725 - recall: 0.2931 - val_loss: 0.4933 - val_accuracy: 0.7585 - val_recall: 0.1849
Epoch 5/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4710 - accuracy: 0.7758 - recall: 0.2935 - val_loss: 0.4900 - val_accuracy: 0.7616 - val_recall: 0.3232
Epoch 6/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4641 - accuracy: 0.7805 

In [19]:
three_month_delinq = dnn(X_train_ct, y_train_2, X_val_ct, y_val_2)

Epoch 1/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4977 - accuracy: 0.7587 - recall: 0.0875 - val_loss: 0.6547 - val_accuracy: 0.5831 - val_recall: 3.8595e-04
Epoch 2/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4852 - accuracy: 0.7657 - recall: 0.2026 - val_loss: 0.6065 - val_accuracy: 0.6460 - val_recall: 0.1949
Epoch 3/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4793 - accuracy: 0.7690 - recall: 0.2193 - val_loss: 0.6480 - val_accuracy: 0.6178 - val_recall: 0.1123
Epoch 4/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4744 - accuracy: 0.7748 - recall: 0.2420 - val_loss: 0.5787 - val_accuracy: 0.7028 - val_recall: 0.4068
Epoch 5/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4691 - accuracy: 0.7771 - recall: 0.2785 - val_loss: 0.6483 - val_accuracy: 0.6502 - val_recall: 0.2242
Epoch 6/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4616 - accuracy: 0.7

In [20]:
six_month_delinq = dnn(X_train_ct, y_train_3, X_val_ct, y_val_3)

Epoch 1/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4960 - accuracy: 0.7603 - recall: 0.1006 - val_loss: 0.6815 - val_accuracy: 0.5893 - val_recall: 0.1747
Epoch 2/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4848 - accuracy: 0.7645 - recall: 0.1959 - val_loss: 0.7133 - val_accuracy: 0.5631 - val_recall: 0.1041
Epoch 3/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4793 - accuracy: 0.7682 - recall: 0.2244 - val_loss: 0.6664 - val_accuracy: 0.5763 - val_recall: 0.1382
Epoch 4/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4738 - accuracy: 0.7730 - recall: 0.2548 - val_loss: 0.7220 - val_accuracy: 0.5835 - val_recall: 0.1520
Epoch 5/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4674 - accuracy: 0.7783 - recall: 0.2646 - val_loss: 0.7278 - val_accuracy: 0.6178 - val_recall: 0.2402
Epoch 6/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4609 - accuracy: 0.7812 